<a href="https://colab.research.google.com/github/myDSMLProjects/Deep-Learning/blob/master/imdb_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [4]:
import tensorflow_datasets as tfds

In [5]:
imdb, info = tfds.load('imdb_reviews' ,with_info=True, as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNV7L2R/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNV7L2R/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteNV7L2R/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [6]:
import numpy as np
train_data, test_data = imdb['train'], imdb['test']

In [7]:
import pandas as pd
pd.DataFrame(train_data)

,0,1
0,"tf.Tensor(b""This was an absolutely terrible mo...","tf.Tensor(0, shape=(), dtype=int64)"
1,tf.Tensor(b'I have been known to fall asleep d...,"tf.Tensor(0, shape=(), dtype=int64)"
2,tf.Tensor(b'Mann photographs the Alberta Rocky...,"tf.Tensor(0, shape=(), dtype=int64)"
3,tf.Tensor(b'This is the kind of film for a sno...,"tf.Tensor(1, shape=(), dtype=int64)"
4,"tf.Tensor(b'As others have mentioned, all the ...","tf.Tensor(1, shape=(), dtype=int64)"
...,...,...
24995,tf.Tensor(b'I have a severe problem with this ...,"tf.Tensor(0, shape=(), dtype=int64)"
24996,"tf.Tensor(b'The year is 1964. Ernesto ""Che"" Gu...","tf.Tensor(1, shape=(), dtype=int64)"
24997,tf.Tensor(b'Okay. So I just got back. Before I...,"tf.Tensor(0, shape=(), dtype=int64)"
24998,tf.Tensor(b'When I saw this trailer on TV I wa...,"tf.Tensor(0, shape=(), dtype=int64)"


In [8]:
training_sentences = []
training_labels = []
testing_sentences = [] 
testing_labels = []

for s, l in train_data:
  training_sentences.append(str(s.numpy()))
  training_labels.append(l.numpy())

for s, l in test_data:
  testing_sentences.append(str(s.numpy()))
  testing_labels.append(l.numpy())

In [9]:
training_sentences[1]

"b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.'"

In [10]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [11]:
vocab_size = 10000
embedding_dim=16
max_length = 120
trunc_type='post'
oov_tok = 'OOV'

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, truncating=trunc_type, maxlen = max_length)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, truncating=trunc_type, maxlen=max_length)

In [12]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(6, activation='relu'),
                             tf.keras.layers.Dense(1, activation='sigmoid')])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
num_epochs=10
model.fit(padded,training_labels_final, validation_data=(testing_padded, testing_labels_final), epochs=num_epochs)

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4882 - accuracy: 0.7528 - val_loss: 0.3809 - val_accuracy: 0.8283
Epoch 2/10
782/782 [==============================] - 7s 9ms/step - loss: 0.2362 - accuracy: 0.9100 - val_loss: 0.4165 - val_accuracy: 0.8182
Epoch 3/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0807 - accuracy: 0.9814 - val_loss: 0.5227 - val_accuracy: 0.8044
Epoch 4/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0178 - accuracy: 0.9982 - val_loss: 0.6170 - val_accuracy: 0.8038
Epoch 5/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0040 - accuracy: 1.0000 - val_loss: 0.6884 - val_accuracy: 0.8080
Epoch 6/10
782/782 [==============================] - 7s 9ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.7424 - val_accuracy: 0.8094
Epoch 7/10
782/782 [==============================] - 7s 9ms/step - loss: 8.0372e-04 - accuracy: 1.0000 - val_loss: 0.7921 - val_accuracy: 0.8092
Ep